# Train YOLOv5 on a Custom Dataset

This tutorial is based on the [YOLOv5 repository](https://github.com/ultralytics/yolov5) by [Ultralytics](https://www.ultralytics.com/). This notebook shows how to train YOLOv5 on a custom dataset prepared in [Roboflow](https://roboflow.com/?ref=studiolab) using [AWS Studio Lab](https://studiolab.sagemaker.aws/). 


### Steps Covered in this Tutorial

To train our detector we take the following steps:

* Install YOLOv5 dependencies
* Download custom YOLOv5 object detection data
* Write our YOLOv5 Training configuration
* Run YOLOv5 training
* Evaluate YOLOv5 performance
* Visualize YOLOv5 training data
* Run YOLOv5 inference on test images
* Export saved YOLOv5 weights for future inference


# Install YOLOv5 Dependencies

## Connecting a GPU

Training will go much faster if you use a GPU Runtime.

From the Studio Lab homepage, select `GPU` as your `Compute type`.

<div><img src="https://i.imgur.com/LHpjx5x.png" style="max-width: 450px;"></div>

To verify that you're running with a GPU, run the following command and ensure that it outputs your GPU stats (if you're running in a CPU runtime, this will give an error message instead):

In [ ]:
!nvidia-smi

If there are no GPUs available, you will still be able to run this notebook, but training time will be greatly increased.

## Cloning the YOLOv5 Repo

Next, we'll pull down [the YOLOv5 repo](https://github.com/ultralytics/yolov5) from Github and install its dependencies.

In [ ]:
# Save the working directory path for later use
import os
HOME = os.getcwd()
print(HOME)

In [ ]:
# clone YOLOv5 repository
!git clone https://github.com/ultralytics/yolov5  # clone repo
%cd yolov5
!git reset --hard fbe67e465375231474a2ad80a4389efc77ecff99

In [ ]:
# install dependencies as necessary
!pip install -qr requirements.txt  # install dependencies (ignore errors)
import torch

from IPython.display import Image, clear_output  # to display images
from utils.downloads import attempt_download  # to download models/datasets

# clear_output()
print('Setup complete. Using torch %s %s' % (torch.__version__, torch.cuda.get_device_properties(0) if torch.cuda.is_available() else 'CPU'))

## Installing Roboflow

We'll be using [Roboflow](https://roboflow.com/?ref=studiolab) to prepare and host our custom object detection dataset (and, optionally, to intelligently sample more images during inference to improve our dataset).

The [`roboflow` pip package](https://blog.roboflow.com/pip-install-roboflow/) will load our dataset in the correct format.

In [ ]:
!pip install roboflow

# Preparing a Custom Dataset

In order to train YOLOv5, we'll need a dataset which is composed of three parts:

1. Images - ideally very similar to the ones our model will make predictions on.
2. Annotations - special TXT files describing bounding boxes that our model will use to learn what it's looking for.
3. A YAML File - contains configuration and metadata needed by YOLOv7 to understand our images and annotations.

Our model will learn to fit the data in the training set, and evaluate its results against the validation set. At the end of the tutorial, we will try our model on the held-out test set to preview how it might perform in the wild when making predictions on images it has never seen before.

We have two options for sourcing our dataset. We can create one from scratch (using our own images, and labeling it with our own annotations), or choose one that's been prepared and open sourced by someone else.



## Option 1: Create a YOLOv5 Dataset with Your Own Images

If you have our own images (and, optionally, annotations), click the three dots to expand the instructions for preparing your own dataset:



_**Note:** If you're running on Sagemaker Studio Lab you will probably want to stop your runtime while you create your dataset so you don't waste your quota. Depending on the size and complexity of your dataset, these steps may take a while to complete._

### Step 1: Sign up for a free Roboflow account

[Roboflow](https://roboflow.com/?ref=studiolab) is an end-to-end computer vision platform. It helps you [create](https://docs.roboflow.com/quick-start?ref=studiolab), [understand](https://blog.roboflow.com/dataset-search/?ref=studiolab), and [use](https://docs.roboflow.com/exporting-data?ref=studiolab) image datasets to train and deploy custom models.

Roboflow strives to be broadly interoperable and can import and export object detection datasets in [dozens of formats](https://roboflow.com/formats?ref=studiolab). They maintain [training notebooks](https://models.roboflow.com/?ref=studiolab) (like this one) for many state of the art computer vision models, and also offer [AutoML training](https://docs.roboflow.com/train?ref=studiolab) which can be useful for [prototyping](https://blog.roboflow.com/deploy-tab/?ref=studiolab), [model assisted labeling](https://roboflow.com/annotate?ref=studiolab), and even [deploying to a wide range of targets and edge devices](https://roboflow.com/deploy?ref=studiolab).

In this tutorial, we will use Roboflow to annotate a custom dataset and export it for use with YOLOv7 in this notebook. But we encourage you to explore [its other features](https://roboflow.com/features?ref=studiolab) as well.

### Step 2: Create a Public Workspace

Roboflow offers a [generous free tier](https://roboflow.com/pricing?ref=studiolab) if your data can be shared publicly with others on [Roboflow Universe](https://universe.roboflow.com/?ref=studiolab). There are also paid plans available for private data.

For this tutorial you'll need to create a Public workspace. Be sure to give it a good name; it will serve as your Universe username where you can showcase your work.

<div><img src="https://i.imgur.com/zfE5MZL.png" style="max-width: 600px;"></div>
    
### Step 3: Create a Project

Then, create an `Object Detection` project (be sure to give it a descriptive name, and fill in the `What will your model predict?` section since they will make your project more understandable and be pulled in via the API later, and can serve as helpful metadata later for advanced use-cases like automated prompt engineering for zero-shot models).

<div><img src="https://i.imgur.com/O2xDyxQ.png" style="max-width: 500px;"></div>

### Step 4: Upload your Images

Next, drop image (or video) files into the UI and, optionally, drop existing annotations in [any supported format](https://roboflow.com/formats?ref=studiolab). Alternatively you can use the [Upload API](https://docs.roboflow.com/adding-data/upload-api?ref=studiolab) or [load images from an S3 bucket](https://blog.roboflow.com/how-to-use-s3-computer-vision-pipeline/?ref=studiolab).

<div><img src="https://i.imgur.com/hWrhtNj.png" style="max-width: 600px;"></div>

Then click `Finish Uploading` to add the images to your Roboflow project.

<div><img src="https://i.imgur.com/9hUh9j5.png" style="max-width: 220px;"></div>

_**Note:** To get good, generalizable results you will need lots of images covering a wide variety of situations and edge cases. Exactly how many images you need [depends on a wide variety of factors](https://blog.roboflow.com/images-train-model/?ref=studiolab), but we recommend starting out with at least 200 for most use-cases. If you need more images, try sourcing from open source datasets on [Roboflow Universe](https://universe.roboflow.com/?ref=studiolab) with images similar to yours._

### Step 5: Annotate

_**Note:** If you imported annotations from another labeling tool or open source dataset, you can skip this step._

Now, we'll use [Roboflow Annotate](https://roboflow.com/annotate?ref=studiolab) to create annotations that will teach our model what we're trying to detect in our images. Since your model will learn to mimic your annotations, it's important that you give some thought to how you label your images ahead of time. We've compiled a list of [best practices to consider when labeling images](https://blog.roboflow.com/tips-for-how-to-label-images/?ref=studiolab).

You can either choose to annotate the images yourself, or invite a friend to your workspace to help out for double the fun.

<video loop autoplay controls src="https://i.imgur.com/AuDAPs4.mp4">Annotate Images</video>


### Step 6: Add to Dataset

Once you have annotated 200 images or more, you're ready to add them to your dataset. At this point you'll choose how to split your images into train, valid, and test sets. We usually recommend keeping the 70/20/10 ratio unless you have more than a few thousand images, in which case you might want to add a higher percentage to your test set.

<div><img src="https://i.imgur.com/XUVE6uq.png" style="max-width: 400px;"></div>

If you're an advanced user, you can also choose to craft these sets by hand to verify that your model is going to generalize well.

### Step 7: View the Health Check (Optional)

Roboflow includes a [dataset health check](https://docs.roboflow.com/dataset-health-check?ref=studiolab) which gives information about your class balance, box distribution, and image sizes. Checking this can help you choose good preprocessing and augmentation steps while generating a version of your dataset which can improve your model's performance.

For example, if you discover your objects are all clustered around the center of the image, you may want to [apply a static crop](https://docs.roboflow.com/image-transformations/image-preprocessing#static-crop) so your model can focus only on the important parts of the image. Or if your images have giant dimensions compared to your objects of interes, [you might want to apply tiling](https://blog.roboflow.com/detect-small-objects/?ref=studiolab).

### Step 8: Generate a Version

Roboflow helps you version control your datasets so that you can get repeatable results & track changes and performance over time. It also lets you preprocess and augment your images which can speed up your training time and improve your model's results.

Choose your preprocessing steps. For YOLOv5, we recommend [Auto-Orient](https://blog.roboflow.com/exif-auto-orientation/?ref=studiolab) and Resize (Stretch to 640x640), which is YOLOv5's default input size. You may also want to [enable Tiling](https://blog.roboflow.com/edge-tiling-during-inference/?ref=studiolab) if your objects are very small compared to the size of your images (if you're not sure, try training a model first without, and if you get poor results you can try again).

<div><img src="https://i.imgur.com/SZojEwP.png" style="max-width: 400px;"></div>

Next, choose your desired augmentations. YOLOv5 does online augmentations during training automatically, so if you have more than 500 annotated images in your dataset you can usually skip this step. But for smaller datasets we've seen good results adding some basic augmentations which mimic things your model may see in the while. We recommend **not** adding Cutout or Mosaic for YOLOv5 as these are already applied during training and applying them twice produces poor results.

Finally, click `Generate` to lock in your choices and render your images.

### Step 9: Export for YOLOv5

Once you've generated a dataset version, click `Export` and select the `YOLO v5 PyTorch` format and the `show download code` option.

<div><img src="https://i.imgur.com/luQC9Rg.png" style="max-width: 500px;"></div>

This will convert your dataset to the proper format and make it available for use in this notebook.

### Step 9: Copy Your Snippet

Now you're all set, simply copy and paste the download snippet from the `Jupyter` tab into the code cell below and you're ready to train your custom YOLOv5 model.

<div><img src="https://i.imgur.com/hZJYgdy.png" style="max-width: 500px;"></div>

## Option 2: Use an Open Source Dataset

[Roboflow Universe](https://universe.roboflow.com/?ref=studiolab) is the world's largest repository of open source datasets. There are [over 100,000 datasets to choose from](https://blog.roboflow.com/computer-vision-datasets-and-apis/?ref=studiolab) for [a plethora of use-cases](https://universe.roboflow.com/browse?ref=studiolab).

If you'd like to start from an open source dataset, click the three dots to expand the instructions:

### Step 1: Find a Dataset

With over 100,000 open source datasets (and over 10,000 of those with a pre-trained model you can try in your browser), it's a near-certainty that you can find a starting point for nearly any computer vision problem on Roboflow Universe.

If you're looking for inspiration, browse [some of the most recently updated projects](https://universe.roboflow.com/search?q=object%20detection%20images%3E%3D100%20images%3C%3D1000%20has%3Amodel&ref=studiolab) or check out some of our [curated collections of datasets](https://universe.roboflow.com/browse?ref=studiolab).

### Step 2: Explore its Contents (Optional)

Once you've found something that looks interesting, you can [explore the images using Dataset Search](https://blog.roboflow.com/dataset-search/?ref=studiolab) to make sure its images cover all the cases you're looking for.

If nothing is quite right, you may want to combine specific images from multiple datasets into a new custom project instead. For example, [the Microsoft COCO Dataset](https://blog.roboflow.com/coco-dataset/?ref=studiolab) doesn't have a class for graffiti, but [it has many images containing graffiti](https://universe.roboflow.com/jacob-solawetz/microsoft-coco/browse?queryText=graffiti&pageSize=50&startingIndex=0&browseQuery=true&ref=studiolab) that you could use to bootstrap a new dataset along with some [other open source graffiti datasets](https://universe.roboflow.com/search?q=graffiti%20object%20detection%20images%3E100&ref=studiolab).

### Step 3: Try a Pre-Trained Model in Your Browser (Optional)

Since many other users of Roboflow Universe have already trained a model on their datasets, you can oftentimes get a sneak preview of how your YOLOv7 model might perform by trying it out in your webcam or on some sample images using [the project's Model tab](https://blog.roboflow.com/deploy-tab/?ref=studiolab).

<video loop autoplay controls src="https://i.imgur.com/yWwbFIs.mp4">Try a Pre-Trained Model in Your Browser</video>

Even though these pre-trained models don't use YOLOv7, oftentimes the limiting factor of prediction quality is in the data, not the model. So they can give you some preliminary intelligence on how well your YOLOv5 model might perform if you train it on that dataset.

### Step 4: Choose a Dataset Version

Many datasets on Roboflow Universe have multiple versions that were generated with different settings (eg different image sizes, and augmentation steps) and, potentially, with different sets of images as the maintainer [added more through active learning](https://blog.roboflow.com/computer-vision-active-learning-tips/?ref=studiolab) over time. 

_**PROTIP:** If multiple versions of the dataset have trained models on Roboflow Universe, you may want to choose the one that achieved the highest mean average precision as it can be a signal of higher._

### Step 5: Get Your Download Snippet

When you've found a dataset version that suits your needs, click `Download` and select the `YOLO v5 PyTorch` format. This will give you a code snippet that will load your dataset into this notebook when you paste it into the code cell below.

<div><img src="https://i.imgur.com/hZJYgdy.png" style="max-width: 500px;"></div>

# Load Dataset in YOLOv5 Format

Now that we have a dataset, we'll download it into our notebook environment in the right format. Use the `YOLO v5 PyTorch` export option in Roboflow.

The YOLOv5 requires YOLO TXT annotations, a custom YAML file, and organized directories. Roboflow creates and hosts this for us and helps download and configure it for our model to consume.

**Copy and paste your code snippet from Roboflow into the code cell below.** The snippet contains a reference to the dataset and an API Key that will authorize you to access your private data from Roboflow (if applicable) and perform (optional) advanced options later.

In [ ]:
%cd {HOME}/yolov5 
#after following the link above, recieve python code with these fields filled in
!pip install roboflow

from roboflow import Roboflow
rf = Roboflow(api_key="YOUR_ROBOFLOW_API_KEY")
project = rf.workspace("joseph-nelson").project("bccd")
dataset = project.version(4).download("yolov5")

In [ ]:
# this is the YAML file Roboflow wrote for us that we're loading into this notebook with our data
%cat {dataset.location}/data.yaml

# Define Model Configuration and Architecture

We will write a yaml script that defines the parameters for our model like the number of classes, anchors, and each layer.

You do not need to edit these cells, but you may.

In [ ]:
# define number of classes based on YAML
import yaml
with open(dataset.location + "/data.yaml", 'r') as stream:
    num_classes = str(yaml.safe_load(stream)['nc'])

In [ ]:
#this is the model configuration we will use for our tutorial 
%cat {HOME}/yolov5/models/yolov5s.yaml #COLAB_EDIT

In [ ]:
#customize iPython writefile so we can write variables
from IPython.core.magic import register_line_cell_magic

@register_line_cell_magic
def writetemplate(line, cell):
    with open(line, 'w') as f:
        f.write(cell.format(**globals()))

In [ ]:
%%writetemplate {HOME}/yolov5/models/custom_yolov5s.yaml

# parameters
nc: {num_classes}  # number of classes
depth_multiple: 0.33  # model depth multiple
width_multiple: 0.50  # layer channel multiple

# anchors
anchors:
  - [10,13, 16,30, 33,23]  # P3/8
  - [30,61, 62,45, 59,119]  # P4/16
  - [116,90, 156,198, 373,326]  # P5/32

# YOLOv5 backbone
backbone:
  # [from, number, module, args]
  [[-1, 1, Focus, [64, 3]],  # 0-P1/2
   [-1, 1, Conv, [128, 3, 2]],  # 1-P2/4
   [-1, 3, BottleneckCSP, [128]],
   [-1, 1, Conv, [256, 3, 2]],  # 3-P3/8
   [-1, 9, BottleneckCSP, [256]],
   [-1, 1, Conv, [512, 3, 2]],  # 5-P4/16
   [-1, 9, BottleneckCSP, [512]],
   [-1, 1, Conv, [1024, 3, 2]],  # 7-P5/32
   [-1, 1, SPP, [1024, [5, 9, 13]]],
   [-1, 3, BottleneckCSP, [1024, False]],  # 9
  ]

# YOLOv5 head
head:
  [[-1, 1, Conv, [512, 1, 1]],
   [-1, 1, nn.Upsample, [None, 2, 'nearest']],
   [[-1, 6], 1, Concat, [1]],  # cat backbone P4
   [-1, 3, BottleneckCSP, [512, False]],  # 13

   [-1, 1, Conv, [256, 1, 1]],
   [-1, 1, nn.Upsample, [None, 2, 'nearest']],
   [[-1, 4], 1, Concat, [1]],  # cat backbone P3
   [-1, 3, BottleneckCSP, [256, False]],  # 17 (P3/8-small)

   [-1, 1, Conv, [256, 3, 2]],
   [[-1, 14], 1, Concat, [1]],  # cat head P4
   [-1, 3, BottleneckCSP, [512, False]],  # 20 (P4/16-medium)

   [-1, 1, Conv, [512, 3, 2]],
   [[-1, 10], 1, Concat, [1]],  # cat head P5
   [-1, 3, BottleneckCSP, [1024, False]],  # 23 (P5/32-large)

   [[17, 20, 23], 1, Detect, [nc, anchors]],  # Detect(P3, P4, P5)
  ]

# Train Custom YOLOv5 Detector

### Next, we'll fire off training!


Here, we are able to pass a number of arguments:
- **img:** define input image size
- **batch:** determine batch size
- **epochs:** define the number of training epochs. (Note: often, 3000+ are common here!)
- **data:** set the path to our yaml file
- **cfg:** specify our model configuration
- **weights:** specify a custom path to weights. (Note: you can download weights from the Ultralytics Google Drive [folder](https://drive.google.com/open?id=1Drs_Aiu7xx6S-ix95f9kNsA6ueKRpN2J))
- **name:** result names
- **nosave:** only save the final checkpoint
- **cache:** cache images for faster training

In [ ]:
# train yolov5s on custom data for 100 epochs
# time its performance
#%%time  #COLAB_EDIT
%cd {HOME}/yolov5/ 
!python train.py --img 416 --batch 16 --epochs 100 --data {dataset.location}/data.yaml --cfg ./models/custom_yolov5s.yaml --weights '' --name yolov5s_results  --cache

# Evaluate Custom YOLOv5 Detector Performance

Training losses and performance metrics are saved to Tensorboard and also to a logfile defined above with the **--name** flag when we train. In our case, we named this `yolov5s_results`. (If given no name, it defaults to `results.txt`.) The results file is plotted as a png after training completes.

Note from Glenn: Partially completed `results.txt` files can be plotted with `from utils.utils import plot_results; plot_results()`.

In [ ]:
from utils.plots import plot_results  # plot results.txt as results.png
Image(filename=HOME+'/yolov5/runs/train/yolov5s_results/results.png', width=1000)  # view results.png

### Curious? Visualize Our Validation Data with Labels

After training starts, view `valid*.jpg` images to see validation images, labels and augmentation effects.

Note a mosaic dataloader is used for training (shown below), a new dataloading concept developed by Glenn Jocher and first featured in [YOLOv4](https://arxiv.org/abs/2004.10934).

In [ ]:
# first, display our ground truth data

print("GROUND TRUTH VALIDATION DATA:")
Image(filename=HOME+'/yolov5/runs/train/yolov5s_results/val_batch0_labels.jpg', width=900)

In [ ]:
# print out an augmented training example
print("GROUND TRUTH AUGMENTED TRAINING DATA:")
Image(filename=HOME+'/yolov5/runs/train/yolov5s_results/train_batch0.jpg', width=900)

#Run Inference  With Trained Weights
Run inference with a pretrained checkpoint on contents of `test/images` folder downloaded from Roboflow.

In [ ]:
# trained weights are saved by default in our weights folder
%ls runs/

In [ ]:
%ls runs/train/yolov5s_results/weights

In [ ]:
# when we ran this, we saw .007 second inference time. That is 140 FPS on a TESLA P100!
# use the best weights!
%cd {HOME}/yolov5
!python detect.py --weights runs/train/yolov5s_results/weights/best.pt --img 416 --conf 0.4 --source {dataset.location}/test/images

In [ ]:
#display inference on ALL test images
#this looks much better with longer training above

import glob
from IPython.display import Image, display

for imageName in glob.glob(HOME+'/yolov5/runs/detect/exp/*.jpg'): #assuming JPG
    display(Image(filename=imageName))
    print("\n")